# Week 5 Problem Set

## Homework

In [2]:
%load_ext nb_mypy
%nb_mypy On

Version 1.0.2


In [3]:
from typing import TypeAlias
from typing import Optional, Any, Iterator
from __future__ import annotations

Number: TypeAlias = int | float
NumberList: TypeAlias = list[int|float] 

**HW1.** Modify the class `TurtleWorld` to include the following attribute and methods:
- `movement_queue` which is an attribute of the type `Queue` to store the movement list. **Each entry of this queue is an object which has the name of the turtle (e.g. "t1") and the movement list (e.g. "ulrr"). The choice of this object is left to you, e.g. it could be a tuple (`("t1", "ulrr")`) or a dictionary with the same information or your own custom class.
- `add_movement(turtle, movement)` which adds turtle movement to the queue `movement_queue` to be run later. The argument `turtle` is a string containing the turtle's name. The argument `movement` is another string for the movement. For example, value for `turtle` can be something like `'t1'` while the value for the `movement` can be something like `'uullrrdd'`.
- `run()` which executes all the movements in the queue.

In [4]:
import math

class Coordinate:
    
    def __init__(self, x:Number=0, y:Number=0) -> None:
        self.x = x
        self.y = y
        
    @property
    def distance(self) -> float:
        return math.sqrt(self.x * self.x + self.y * self.y)
    
    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

In [5]:
class Queue:
    def __init__(self) -> None:
        self.__items: list[Any] = []
    
    def enqueue(self, item: Any) -> None:
        self.__items.append(item)

    
    def dequeue(self) -> Any:
        if self.is_empty:
            return None
        return self.__items.pop(0)
    
    def peek(self) -> Any:
        if self.is_empty:
            return None
        return self.__items[0]
    
    @property
    def is_empty(self) -> bool:
        return len(self.__items) == 0

    
    @property
    def size(self) -> int:
        return len(self.__items)

In [6]:
# Class definition
class RobotTurtle:
    # Attributes:
    def __init__(self, name: str, speed: int=1) -> None:
        self.name: str = name
        self.speed: int = speed
        self._pos: Coordinate = Coordinate(0, 0)
        
    @property
    def name(self) -> str:
        return self._name
    
    @name.setter
    def name(self, value: str) -> None:
        if isinstance(value, str) and value != "":
            self._name: str = value
            
    @property
    def speed(self) -> int:
        return self._speed
    
    @speed.setter
    def speed(self, value: int) -> None:
        if isinstance(value, int) and value > 0:
            self._speed: int = value

    @property
    def pos(self) -> Coordinate:
        return self._pos
    
    def move(self, direction: str) -> None:
        update: dict[str, Coordinate] = {'up' : Coordinate(self.pos.x, self.pos.y + self.speed),
                                        'down' : Coordinate(self.pos.x, self.pos.y - self.speed),
                                        'left' : Coordinate(self.pos.x - self.speed, self.pos.y),
                                        'right' : Coordinate(self.pos.x + self.speed, self.pos.y)}
        self._pos = update[direction]

        
    def tell_name(self) -> None:
        print(f"My name is {self.name}")


In [7]:
class TurtleWorld:
    valid_movements: set[str] = set('udlr')
    movement_map: dict[str, str] = {'u': 'up', 'd': 'down', 'l': 'left', 'r': 'right'}
    
    def __init__(self) -> None:
        self.turtles: dict[str, RobotTurtle] = {}
        self.movement_queue = Queue()
        
    def add_movement(self, turtle: str, movement: str) -> None:
        if turtle in self.turtles:
            valid_movement = ''.join([m for m in movement if m in self.valid_movements])
            if valid_movement:
                self.movement_queue.enqueue((turtle, valid_movement))
    
    def run(self) -> None:
        while not self.movement_queue.is_empty:
            turtle, movement = self.movement_queue.dequeue()
            self.move_turtle(turtle, movement)
        
    def move_turtle(self, name: str, movement: str) -> None:
        if name in self.turtles:
            for move in movement:
                self.turtles[name].move(self.movement_map[move])
    
    def add_turtle(self, name: str, speed: int) -> None:
        self.turtles.update({name: RobotTurtle(name, speed)})
        
    def remove_turtle(self, name: str) -> None:
        self.turtles.pop(name)
        
    def list_turtles(self) -> list[str]:
        return sorted(list(self.turtles.keys()))


In [8]:
world: TurtleWorld = TurtleWorld()
assert isinstance(world.movement_queue, Queue)

world.add_turtle('t1', 1)
world.add_turtle('t2', 2)
world.add_movement('t1', 'ur')
world.add_movement('t2', 'urz')
assert str(world.turtles['t1'].pos) == '(0, 0)'
assert str(world.turtles['t2'].pos) == '(0, 0)'
assert world.movement_queue.size == 2

world.run()
assert str(world.turtles['t1'].pos) == '(1, 1)'
assert str(world.turtles['t2'].pos) == '(2, 2)'

world.add_movement('t1', 'ur')
world.add_movement('t2', 'urz')

world.run()
assert str(world.turtles['t1'].pos) == '(2, 2)'
assert str(world.turtles['t2'].pos) == '(4, 4)'


###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [9]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Implement a radix sorting machine. A radix sort for base 10 integers is a *mechanical* sorting technique that utilizes a collection of bins:
- one main bin 
- 10 digit-bins

Each bin acts like a *queue* and maintains its values in the order that they arrive. The algorithm works as follows:
- it begins by placing each number in the main bin. 
- Then it considers each value digit by digit. The first value is removed from the main bin and placed in a digit-bin corresponding to the digit being considered. For example, if the ones digit is being considered, 534 will be placed into digit-bin 4 and 667 will placed into digit-bin 7. 
- Once all the values are placed into their corresponding digit-bins, the values are collected from bin 0 to bin 9 and placed back in the main bin (in that order). 
- The process continues with the tens digit, the hundreds, and so on. 
- After the last digit is processed, the main bin will contain the values in ascending order.

Create a class `RadixSort` that takes in a List of Integers during object instantiation. The class should have the following properties:
- `items`: is a List of Integers containing the numbers.

It should also have the following methods:
- `sort()`: which returns the sorted numbers from `items` as an `list` of Integers.
- `max_digit()`: which returns the maximum number of digits of all the numbers in `items`. For example, if the numbers are 101, 3, 1041, this method returns 4 as the result since the maximum digit is four from 1041. 
- `convert_to_str(items)`: which returns items as a list of Strings (instead of Integers). This function should pad the higher digits with 0 when converting an Integer to a String. For example if the maximum digit is 4, the following items are converted as follows. From `[101, 3, 1041]` to `["0101", "0003", "1041"]`.

Hint: Your implementation should make use of the generic `Queue` class, which you created, for the bins.

In [10]:
class RadixSort:
    
    def __init__(self, my_list: list[int]) -> None:
        self.items = my_list
    
    def max_digit(self) -> int:
        return max(len(str(item)) for item in self.items)
    
    def convert_to_str(self, items: list[int]) -> list[str]:
        max_len = self.max_digit()
        return [f"{item:0{max_len}d}" for item in items] #0{max_len}: This part specifies that the number should be zero-padded to a width                                                                         of max_len characters.
                                                        #d: This indicates that the value should be formatted as a decimal integer.
    
    def sort(self) -> list[int]:
        max_len = self.max_digit()
        bins = [Queue() for i in range(10)]
        
        for digit in range(max_len - 1, -1, -1):
            for item in self.convert_to_str(self.items):
                bins[int(item[digit])].enqueue(int(item))
            
            self.items = []
            for bin in bins:
                while not bin.is_empty:
                    self.items.append(bin.dequeue())
        
        return self.items

In [11]:
list1: RadixSort = RadixSort([101, 3, 1041])
assert list1.items == [101,3,1041]
assert list1.max_digit() == 4
assert list1.convert_to_str(list1.items) == ["0101", "0003", "1041"]
ans: list[int] = list1.sort()
print(ans)
assert ans == [3, 101, 1041]
list2: RadixSort = RadixSort([23, 1038, 8, 423, 10, 39, 3901])
assert list2.sort() == [8, 10, 23, 39, 423, 1038, 3901]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[3, 101, 1041]


In [12]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** Write a class called `EvaluateFraction` that evaluates postfix notation implemented using Stack and Queue data structures. Postfix notation is a way of writing expressions without using parenthesis. For example, the expression `(1+2)*3` would be written as `1 2 + 3 *`. The class `EvaluateFraction` has the following method:
- `input(inp)`: which pushes the input input one at a time. For example, to create a postfix notation `1 2 + 3 *`, we can call this method repetitively, e.g. `e.input('1'); e.input('2'); e.input('+'); e.input('3'); e.input('*')`. Notice that the input is of String data type. 
- `evaluate()`: which returns the output of the expression.
- `get_fraction(inp)`: which takes in an input string and returns a `Fraction` object. 

Postfix notation is evaluated using a Stack. The input streams from `input()` are stored in a Queue. If the output of the Queue is a number, the item is pushed into the stack. If it is an operator, we will apply the operator to the two top most item n the stacks and push the result back into the stack. 

In [13]:
class Stack:
    def __init__(self) -> None:
        self.__items: list[Any] = []

        
    def push(self, item: Any):
        self.__items.append(item)

    def pop(self) -> Any:
        if not self.is_empty:
            return self.__items.pop()
        return None

    def peek(self) -> Any:
        if not self.is_empty:
            return self.__items[-1]
        return None

    @property
    def is_empty(self) -> bool:
        return len(self.__items) == 0

    @property
    def size(self):
        return len(self.__items)

In [14]:
class Queue:
    def __init__(self) -> None:
        self.left_stack: Stack = Stack()
        self.right_stack: Stack = Stack()
    
    def enqueue(self, item: Any) -> None:
        self.left_stack.push(item)

    def dequeue(self) -> Any:
        if self.right_stack.is_empty:
            while not self.left_stack.is_empty:
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.pop()

    def peek(self) -> Any:
        if self.right_stack.is_empty:
            while not self.left_stack.is_empty:
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.peek()
    
    @property
    def is_empty(self) -> bool:
        return self.left_stack.is_empty and self.right_stack.is_empty
        
    @property
    def size(self) -> int:
        return self.left_stack.size + self.right_stack.size


In [15]:
def gcd(a: int, b: int) -> int:
    if b == 0:
        return a
    else:
        return gcd(b, a % b)


class Fraction:
    def __init__(self, num: int, den: int) -> None:
        self.num = num
        self.den = den if den != 0 else 1
    
    @property
    def num(self) -> int:
        return self._num
    
    @num.setter
    def num(self, val: int) -> None:
        if isinstance(val, (int, float)):
            self._num = int(val)
        else:
            print("Numerator must be an integer or float")
    
    @property
    def den(self) -> int:
        return self._den
    
    @den.setter
    def den(self, val: int) -> None:
        if isinstance(val, (int, float)):
            val = int(val)
            if val == 0:
                self._den = 1
            else:
                self._den = val
        else:
            print("Denominator must be an integer or float")
    
    def __str__(self) -> str:
        return f"{self.num}/{self.den}"    
    
    def simplify(self) -> Fraction:
        common_divisor = gcd(self.num,self.den)
        return Fraction(self.num//common_divisor,self.den//common_divisor)
    
    def __add__(self, other) -> Fraction:
        new_num = self.num*other.den + other.num*self.den
        new_den = self.den*other.den
        return Fraction (new_num,new_den).simplify()
        
    
    def __eq__(self, other) -> bool:
        simp_self = self.simplify()
        simp_other = other.simplify()
        return (simp_self.den == simp_other.den and simp_self.num ==simp_other.num)
    

    
    def __sub__(self, other) -> Fraction:
        new_num = self.num * other.den - other.num * self.den
        new_den = self.den * other.den
        return Fraction(new_num, new_den).simplify()
    
    def __mul__(self, other) -> Fraction:
        new_num = self.num * other.num
        new_den = self.den * other.den
        return Fraction(new_num, new_den).simplify()
    
    def __lt__(self, other) -> bool:
        return self.num * other.den < other.num * self.den

    
    def __le__(self, other) -> bool:
        return self.num * other.den <= other.num * self.den
    
    def __gt__(self, other) -> bool:
        return self.num * other.den > other.num * self.den

    
    def __ge__(self, other) -> bool:
        return self.num * other.den >= other.num * self.den



In [23]:
class EvaluateFraction:

    operands: str = "0123456789"
    operators: str = "+-*/"
    
    def __init__(self) -> None:
        self.expression: Queue = Queue()
        self.stack: Stack = Stack()
    
    def input(self, item: str) -> None:
        self.expression.enqueue(item)
    
    def evaluate(self) -> Fraction:
        while not self.expression.is_empty:
            item = self.expression.dequeue()
            if '/' in item:
                self.stack.push(self.get_fraction(item))
            elif item in self.operators:
                op2 = self.stack.pop()
                op1 = self.stack.pop()
                result = self.process_operator(op1, op2, item)
                self.stack.push(result)
        return self.stack.pop()
    
    def get_fraction(self, inp: str) -> Fraction:
        num, den = map(int, inp.split('/'))
        return Fraction(num, den)
    
    def process_operator(self, op1: Fraction, op2: Fraction, op: str) -> Fraction:
        if op == '+':
            return op1 + op2
        elif op == '-':
            return op1 - op2
        elif op == '*':
            return op1 * op2
        elif op == '/':
            return Fraction(op1.num * op2.den, op1.den * op2.num).simplify()
        else:
            raise ValueError(f"Unknown operator: {op}")



In [24]:
pe: EvaluateFraction = EvaluateFraction()
pe.input("1/2")
pe.input("2/3")
pe.input("+")
assert pe.evaluate()==Fraction(7, 6)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
assert pe.evaluate()==Fraction(1, 1)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
pe.input("3/4")
pe.input("*")
assert pe.evaluate()==Fraction(3, 4)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [25]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** Modify HW2 so that it can work with MixedFraction. Write a class called `EvaluateMixedFraction` as a subclass of `EvaluateFraction`. You need to override the following methods:
- `get_fraction(inp)`: This function should be able to handle string input for MixedFraction such as `1 1/2` or `3/2`. It should return a `MixedFraction` object.
- `evaluate()`: This function should return `MixedFraction` object rather than `Fraction` object. 

In [26]:
class MixedFraction(Fraction):
    def __init__(self, top: int, bot: int, whole: int=0) -> None:
        num = whole * bot + top
        super().__init__(num, bot)

    def get_three_numbers(self) -> tuple[int, int, int]:
        whole = self.num // self.den
        top = self.num % self.den
        bot = self.den
        return (top, bot, whole)

    def __str__(self) -> str:
        top, bot, whole = self.get_three_numbers()
        if whole == 0:
            return f"{top}/{bot}"
        else:
            return f"{whole} {top}/{bot}"



In [27]:
class EvaluateMixedFraction(EvaluateFraction):
    def get_fraction(self, inp: str) -> Fraction:
        if ' ' in inp:
            whole, frac = inp.split(' ')
            num, den = map(int, frac.split('/'))
            return MixedFraction(num, den, int(whole))
        else:
            num, den = map(int, inp.split('/'))
            return MixedFraction(num, den)
    
    def evaluate(self) -> MixedFraction:
        while not self.expression.is_empty:
            item = self.expression.dequeue()
            if '/' in item:
                self.stack.push(self.get_fraction(item))
            elif item in self.operators:
                op2 = self.stack.pop()
                op1 = self.stack.pop()
                result = self.process_operator(op1, op2, item)
                self.stack.push(result)
        answer = self.stack.pop()
        return MixedFraction(answer.num, answer.den)

In [28]:
pe: EvaluateMixedFraction = EvaluateMixedFraction()
pe.input("3/2")
pe.input("1 2/3")
pe.input("+")
out: MixedFraction = pe.evaluate() 
assert out == MixedFraction(1, 6, 3)
assert isinstance(out, MixedFraction)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1 1/8")
pe.input("-")
assert pe.evaluate() == MixedFraction(1, 24)

pe.input("1 1/2")
pe.input("2 2/3")
pe.input("+")
pe.input("1 1/6")
pe.input("-")
pe.input("5/4")
pe.input("*")
assert pe.evaluate() == MixedFraction( 3, 4, 3)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [29]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** Write a function that takes in a graph representation in a dictionary and convert it to an object-oriented representation using the `Graph` class. Refer to your cohort problem sets for the `Graph` and `Vertex` class definition. 

The function takes in a dictionary that has the following format:
```python
graph = {'vertex1': [neighbour1, neighbour2, ...],
         'vertex2': [neighbour1, ...]}
```

Notes:
- The graph is represented as and adjecancy list. 
- The key of the dictionary is the `id` of the vertex in the graph. 
- The value of the dictionary is a list of neighbours of that vertex.
- The element in the list of neighbours is the `id` of the neighbouring vertices. 
- The data type for `id` is a string. 

Example:
```python
graph = {"A": ["B", "C"], 
         "B": ["C", "D"],
         "C": ["D"],
         "D": ["C"], 
         "E": ["F"],
         "F": ["C"]}
```

In [30]:
class Vertex:
    def __init__(self, id_: str="") -> None:
        self.id_: str = id_
        self.neighbours: dict[Vertex, Number] = {}
    
    def add_neighbour(self, nbr_vertex: Vertex, weight: Number=0) -> None:
        self.neighbours[nbr_vertex] = weight
    
    def get_neighbours(self) -> list[Vertex]:
        return list(self.neighbours.keys())
    
    def get_weight(self, neighbour: Vertex) -> Optional[Number]:
        return self.neighbours.get(neighbour, None)
    
    def __eq__(self, other) -> bool:
        return self.id_ == other.id_
    
    def __lt__(self, other) -> bool:
        return self.id_ < other.id_
    
    def __hash__(self) -> int:
        return hash(self.id_)
    
    def __str__(self) -> str:
        neighbours_ids = ', '.join([nbr.id_ for nbr in self.neighbours])
        return f"Vertex {self.id_} is connected to: {neighbours_ids}"

In [31]:
class Graph:
    def __init__(self) -> None:
        self.vertices: dict[str, Vertex] = {}
        
    @property
    def num_vertices(self) -> int:
        return len(self.vertices)

    def _create_vertex(self, id_: str) -> Vertex:
        return Vertex(id_)

    def add_vertex(self, id_: str) -> None:
        if id_ not in self.vertices:
            self.vertices[id_] = self._create_vertex(id_)

    def get_vertex(self, id_: str) -> Optional[Vertex]:
        return self.vertices.get(id_, None)

    def add_edge(self, start_v: str, end_v: str, weight: Number=0) -> None:
        if start_v not in self.vertices:
            self.add_vertex(start_v)
        if end_v not in self.vertices:
            self.add_vertex(end_v)
        self.vertices[start_v].add_neighbour(self.vertices[end_v], weight)

    def get_neighbours(self, id_: str) -> list[str]:
        vertex = self.get_vertex(id_)
        if vertex:
            return [nbr.id_ for nbr in vertex.get_neighbours()]
        return []
    
    def __contains__(self, val: str) -> bool:
        return val in self.vertices.keys()
    
    def __iter__(self):
        for k,v in self.vertices.items():
            yield v 


In [32]:
def create_graph_object(g: dict[str, list[str]]) -> Graph:
    output: Graph = Graph()
    for vertex, neighbours in g.items():
        output.add_vertex(vertex)
        for neighbour in neighbours:
            output.add_edge(vertex, neighbour)
    return output

In [33]:
graph: dict[str, list[str]] = {"A": ["B", "C"], 
         "B": ["C", "D"],
         "C": ["D"],
         "D": ["C"], 
         "E": ["F"],
         "F": ["C"]}

output: Graph = create_graph_object(graph)
assert output.num_vertices == 6
assert sorted([x.id_ for x in output]) == ["A", "B", "C", "D", "E", "F"]
assert sorted(output.get_neighbours("A")) == ["B", "C"]
assert sorted(output.get_neighbours("C")) == ["D"]
assert sorted(output.get_neighbours("E")) == ["F"]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [51]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###
